## Prepare the environment

In [1]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Thu Dec  7 09:54:20 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 546.17                 Driver Version: 546.17       CUDA Version: 12.3     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                     TCC/WDDM  | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce RTX 2070 ...  WDDM  | 00000000:01:00.0  On |                  N/A |
| 25%   41C    P8              21W / 215W |   1001MiB /  8192MiB |     11%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

## Update the unix package

In [2]:
!add-apt-repository -y ppa:jonathonf/ffmpeg-4
!apt update
!apt install -y ffmpeg

'add-apt-repository' is not recognized as an internal or external command,
operable program or batch file.
'apt' is not recognized as an internal or external command,
operable program or batch file.
'apt' is not recognized as an internal or external command,
operable program or batch file.


## Install pip dependencies

In [3]:
!pip install datasets>=2.6.1
!pip install git+https://github.com/huggingface/transformers
!pip install librosa
!pip install evaluate>=0.30
!pip install jiwer
!pip install gradio
!pip install hopsworks

  Cloning https://github.com/huggingface/transformers to c:\users\ merlijn\appdata\local\temp\pip-req-build-h7fcs23x
  Resolved https://github.com/huggingface/transformers to commit 44b5506d29326c6f2ae672e78d2df78ec55939d9
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'


  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers 'C:\Users\ merlijn\AppData\Local\Temp\pip-req-build-h7fcs23x'


## Log into huggingface

##Log into hopsworks

In [4]:
import hopsworks
project = hopsworks.login()

C:\Users\ merlijn\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/194708


## Load Dataset

In [5]:
from datasets import load_dataset, DatasetDict

common_voice = DatasetDict()

common_voice["train"] = load_dataset("mozilla-foundation/common_voice_11_0", "it", split="train+validation", use_auth_token=True)
common_voice["test"] = load_dataset("mozilla-foundation/common_voice_11_0", "it", split="test", use_auth_token=True)

print(common_voice)

PydanticDeprecatedSince20: Pydantic V1 style `@validator` validators are deprecated. You should migrate to Pydantic V2 style `@field_validator` validators, see the migration guide for more details. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.5/migration/
PydanticDeprecatedSince20: Pydantic V1 style `@validator` validators are deprecated. You should migrate to Pydantic V2 style `@field_validator` validators, see the migration guide for more details. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.5/migration/
PydanticDeprecatedSince20: Pydantic V1 style `@validator` validators are deprecated. You should migrate to Pydantic V2 style `@field_validator` validators, see the migration guide for more details. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.5/migration/
PydanticDeprecatedS

DatasetDict({
    train: Dataset({
        features: ['client_id', 'path', 'audio', 'sentence', 'up_votes', 'down_votes', 'age', 'gender', 'accent', 'locale', 'segment'],
        num_rows: 167606
    })
    test: Dataset({
        features: ['client_id', 'path', 'audio', 'sentence', 'up_votes', 'down_votes', 'age', 'gender', 'accent', 'locale', 'segment'],
        num_rows: 15003
    })
})


# We remove accents, gender, age and other features that are not relevant for our purpose

In [6]:
common_voice = common_voice.remove_columns(["accent", "age", "client_id", "down_votes", "gender", "locale", "path", "segment", "up_votes"])

print(common_voice)

DatasetDict({
    train: Dataset({
        features: ['audio', 'sentence'],
        num_rows: 167606
    })
    test: Dataset({
        features: ['audio', 'sentence'],
        num_rows: 15003
    })
})


## Prepare Feature Extractor, Tokenizer and Data

### Load WhisperFeatureExtractor

In [7]:
from transformers import WhisperFeatureExtractor

feature_extractor = WhisperFeatureExtractor.from_pretrained("openai/whisper-small")

### Load WhisperTokenizer

In [8]:
from transformers import WhisperTokenizer

tokenizer = WhisperTokenizer.from_pretrained("openai/whisper-small", language="Italian", task="transcribe")

### Combine To Create A WhisperProcessor

In [9]:
from transformers import WhisperProcessor

processor = WhisperProcessor.from_pretrained("openai/whisper-small", language="Italian", task="transcribe")

### Prepare Data

In [10]:
# from datasets import Audio

# common_voice = common_voice.cast_column("audio", Audio(sampling_rate=16000))

In [11]:
# def prepare_dataset(batch, feature_extractor, tokenizer):
#     # compute log-Mel input features from input audio array
#     batch["input_features"] = feature_extractor(batch["audio"]["array"], sampling_rate=batch["audio"]["sampling_rate"]).input_features[0]

#     # Assuming you have a tokenizer defined somewhere
#     # Replace 'tokenizer' with the actual name of your tokenizer
#     batch["labels"] = tokenizer(batch["sentence"]).input_ids
#     return batch


# # Assuming 'common_voice' is defined somewhere in your code
# # Replace 'common_voice.column_names["train"]' with the actual columns you want to remove
# common_voice = common_voice.map(
#     lambda batch: prepare_dataset(batch, feature_extractor, tokenizer),
#     remove_columns=common_voice.column_names["train"],
# )

## Get information about the voices

In [12]:
# help(common_voice)

In [13]:
# Save the dataset locally
#common_voice.save_to_disk("common_voice")

## If we want to load from the disk

In [14]:
from datasets import load_dataset, DatasetDict

common_voice = DatasetDict.load_from_disk("common_voice")

## Get all the files on the disk

In [15]:
import os
print(os.getcwd())
print(os.listdir("./common_voice/"))
print(os.listdir("./common_voice/train"))
print(os.listdir("./common_voice/test"))

c:\Users\ merlijn\Documents\GitHub\lab2sml
['dataset_dict.json', 'test', 'train']
['data-00000-of-00322.arrow', 'data-00001-of-00322.arrow', 'data-00002-of-00322.arrow', 'data-00003-of-00322.arrow', 'data-00004-of-00322.arrow', 'data-00005-of-00322.arrow', 'data-00006-of-00322.arrow', 'data-00007-of-00322.arrow', 'data-00008-of-00322.arrow', 'data-00009-of-00322.arrow', 'data-00010-of-00322.arrow', 'data-00011-of-00322.arrow', 'data-00012-of-00322.arrow', 'data-00013-of-00322.arrow', 'data-00014-of-00322.arrow', 'data-00015-of-00322.arrow', 'data-00016-of-00322.arrow', 'data-00017-of-00322.arrow', 'data-00018-of-00322.arrow', 'data-00019-of-00322.arrow', 'data-00020-of-00322.arrow', 'data-00021-of-00322.arrow', 'data-00022-of-00322.arrow', 'data-00023-of-00322.arrow', 'data-00024-of-00322.arrow', 'data-00025-of-00322.arrow', 'data-00026-of-00322.arrow', 'data-00027-of-00322.arrow', 'data-00028-of-00322.arrow', 'data-00029-of-00322.arrow', 'data-00030-of-00322.arrow', 'data-00031-of-003

## Get the total size of the training data

In [16]:
def get_dir_size(path='./common_voice/train'):
    total = 0
    with os.scandir(path) as it:
        for entry in it:
            if entry.is_file():
                total += entry.stat().st_size
            elif entry.is_dir():
                total += get_dir_size(entry.path)
    return total
sz = get_dir_size()
print(sz)

160990336699


### Define a Data Collator

In [17]:
import torch

from dataclasses import dataclass
from typing import Any, Dict, List, Union

@dataclass
class DataCollatorSpeechSeq2SeqWithPadding:
    processor: Any

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lengths and need different padding methods
        # first treat the audio inputs by simply returning torch tensors
        input_features = [{"input_features": feature["input_features"]} for feature in features]
        batch = self.processor.feature_extractor.pad(input_features, return_tensors="pt")

        # get the tokenized label sequences
        label_features = [{"input_ids": feature["labels"]} for feature in features]
        # pad the labels to max length
        labels_batch = self.processor.tokenizer.pad(label_features, return_tensors="pt")

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        # if bos token is appended in previous tokenization step,
        # cut bos token here as it's append later anyways
        if (labels[:, 0] == self.processor.tokenizer.bos_token_id).all().cpu().item():
            labels = labels[:, 1:]

        batch["labels"] = labels

        return batch

In [18]:
data_collator = DataCollatorSpeechSeq2SeqWithPadding(processor=processor)

### Define Evaluation Metrics

In [19]:
import evaluate

metric = evaluate.load("wer")

2023-12-07 09:55:36,484 WARNING: From C:\Users\ merlijn\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\keras\src\losses.py:2976: The name tf.losses.sparse_softmax_cross_entropy is deprecated. Please use tf.compat.v1.losses.sparse_softmax_cross_entropy instead.



In [20]:
def compute_metrics(pred):
    pred_ids = pred.predictions
    label_ids = pred.label_ids

    # replace -100 with the pad_token_id
    label_ids[label_ids == -100] = tokenizer.pad_token_id

    # we do not want to group tokens when computing the metrics
    pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    label_str = tokenizer.batch_decode(label_ids, skip_special_tokens=True)

    wer = 100 * metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}

## Load a pre trained checkpoint

In [21]:
from transformers import WhisperForConditionalGeneration

model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-small")

In [22]:
model.config.forced_decoder_ids = None
model.config.suppress_tokens = []

### Define the Training Configuration

In [30]:
from transformers import Seq2SeqTrainingArguments

training_args = Seq2SeqTrainingArguments(
    num_train_epochs=1,
    output_dir="./whisper-small-hi",  # change to a repo name of your choice
    per_device_train_batch_size=16,
    gradient_accumulation_steps=1,  # increase by 2x for every 2x decrease in batch size
    learning_rate=1e-5,
    warmup_steps=50,
    max_steps=200,
    gradient_checkpointing=True,
    fp16=True,
    evaluation_strategy="steps",
    per_device_eval_batch_size=8,
    predict_with_generate=True,
    generation_max_length=225,
    save_steps= 50,
    eval_steps= 50,
    logging_steps=25,
    report_to=["tensorboard"],
    load_best_model_at_end=True,
    metric_for_best_model="wer",
    greater_is_better=False,
    push_to_hub=True,
)

In [31]:
from transformers import Seq2SeqTrainer

# Train the model using a small part of the training set
common_voice_train = common_voice["train"].select(range(1000))
common_voice_test = common_voice["test"].select(range(200))

trainer = Seq2SeqTrainer(
    args=training_args,
    model=model,
    train_dataset=common_voice_train,
    eval_dataset=common_voice_test,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=processor.feature_extractor,
)

In [32]:
processor.save_pretrained(training_args.output_dir)

In [26]:
trainer.train()

  0%|          | 0/100 [00:00<?, ?it/s]UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
`use_cache = True` is incompatible with gradient checkpointing. Setting `use_cache = False`...
 25%|██▌       | 25/100 [20:39<1:17:35, 62.07s/it]

{'loss': 2.7882, 'learning_rate': 9.875000000000001e-06, 'epoch': 0.4}


 50%|█████     | 50/100 [39:16<39:30, 47.41s/it]  

{'loss': 0.991, 'learning_rate': 6.750000000000001e-06, 'epoch': 0.79}


 75%|███████▌  | 75/100 [55:24<14:40, 35.22s/it]

{'loss': 0.7143, 'learning_rate': 3.625e-06, 'epoch': 1.19}


100%|██████████| 100/100 [1:10:59<00:00, 42.60s/it]

{'loss': 0.5545, 'learning_rate': 5.000000000000001e-07, 'epoch': 1.59}
{'train_runtime': 4259.5013, 'train_samples_per_second': 0.376, 'train_steps_per_second': 0.023, 'train_loss': 1.2620051860809327, 'epoch': 1.59}


TrainOutput(global_step=100, training_loss=1.2620051860809327, metrics={'train_runtime': 4259.5013, 'train_samples_per_second': 0.376, 'train_steps_per_second': 0.023, 'train_loss': 1.2620051860809327, 'epoch': 1.59})

In [27]:
# print(torch.cuda.memory_summary(device=None, abbreviated=False))
# import torch
# torch.cuda.empty_cache()

In [28]:
kwargs = {
    "dataset_tags": "mozilla-foundation/common_voice_11_0",
    "dataset": "Common Voice 11.0",  # a 'pretty' name for the training dataset
    "dataset_args": "config: hi, split: test",
    "language": "Italian",
    "model_name": "Whisper Small - Italian",  # a 'pretty' name for our model
    "finetuned_from": "openai/whisper-small",
    "tasks": "automatic-speech-recognition",
    "tags": "hf-asr-leaderboard",
}

In [36]:
# trainer.push_to_hub(**kwargs)

# Get the model from the hub

model = WhisperForConditionalGeneration.from_pretrained("whisper-small-hi")

In [37]:
from transformers import Seq2SeqTrainer

# Train the model using a small part of the training set
common_voice_train = common_voice["train"].select(range(1000))
common_voice_test = common_voice["test"].select(range(200))

trainer = Seq2SeqTrainer(
    args=training_args,
    model=model,
    train_dataset=common_voice_train,
    eval_dataset=common_voice_test,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=processor.feature_extractor,
)

In [38]:
trainer.train()

  0%|          | 3/2000 [02:50<33:10:00, 59.79s/it]

KeyboardInterrupt: 